In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pwd

'/content'

In [ ]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!pip install tensorflow scikit-learn numpy mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 33.0 MB/s eta 0:00:00


In [ ]:
import numpy as np
import os
import cv2
import mediapipe as mp
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import tensorflow as tf

# MediaPipe holistic 모델 초기화
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# 한국어 단어 목록
actions = ['곰', '놀이터', '다리', '바다', '벌', '병원', '선생님', '엄마', '유치원', '학교']
sequence_length = 30
DATA_PATH = 'KSL_MP_2'

# 한 프레임당 랜드마크 추출 및 저장
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in
                     results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33 * 4)
    lh = np.array([[res.x, res.y, res.z] for res in
                   results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21 * 3)
    rh = np.array([[res.x, res.y, res.z] for res in
                   results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21 * 3)
    return np.concatenate([pose, lh, rh])

# 비디오 파일 처리 및 랜드마크 저장
def process_videos(data_path, actions, sequence_length):
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for action in actions:
            action_path = os.path.join(data_path, action)
            if not os.path.exists(action_path):
                continue
            videos = [video for video in os.listdir(action_path) if video.endswith(".mp4")]
            for video in videos:
                video_path = os.path.join(action_path, video)
                cap = cv2.VideoCapture(video_path)
                length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
                fps = cap.get(cv2.CAP_PROP_FPS)
                frame_duration = length / fps
                frames_to_capture = np.linspace(0, frame_duration, sequence_length, endpoint=False)

                for i, frame_time in enumerate(frames_to_capture):
                    cap.set(cv2.CAP_PROP_POS_MSEC, frame_time * 1000)
                    ret, frame = cap.read()
                    if not ret:
                        break
                    image, results = mediapipe_detection(frame, holistic)
                    keypoints = extract_keypoints(results)
                    npy_path = os.path.join(DATA_PATH, action, video.split('.')[0])
                    os.makedirs(npy_path, exist_ok=True)
                    np.save(os.path.join(npy_path, f"{i}.npy"), keypoints)
                cap.release()

# 이미지 처리 함수
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

# 메인 데이터 처리
process_videos('KSL', actions, sequence_length)

# 데이터 증강
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, labels, batch_size, shuffle=True, augment=False):
        self.data = data
        self.labels = labels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.data) / self.batch_size))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_data = [self.data[k] for k in indexes]
        batch_labels = [self.labels[k] for k in indexes]

        X, y = self.__data_generation(batch_data, batch_labels)
        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.data))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, batch_data, batch_labels):
        X = np.array(batch_data)
        y = np.array(batch_labels)

        # 증강 적용
        if self.augment:
            X = self.add_noise(X)
            X = self.time_shift(X)
        return X, y

    def add_noise(self, data, noise_factor=0.1):
        noise = np.random.normal(loc=0.0, scale=noise_factor, size=data.shape)
        data_noisy = data + noise
        return data_noisy

    def time_shift(self, data, max_shift_ratio=0.05):
        shifted_data = []
        for sequence in data:
            sequence_length = sequence.shape[0]
            shift_amount = np.random.randint(int(sequence_length * max_shift_ratio))
            if np.random.rand() > 0.5:
                shifted_sequence = np.roll(sequence, shift_amount, axis=0)
            else:
                shifted_sequence = np.roll(sequence, -shift_amount, axis=0)
            shifted_data.append(shifted_sequence)
        return np.array(shifted_data)

# 데이터 로드 및 전처리
actions = ['곰', '놀이터', '다리', '바다', '벌', '병원', '선생님', '엄마', '유치원', '학교']
DATA_PATH = 'KSL_MP_2'
sequence_length = 30
data = []
labels = []

for action in actions:
    action_folder = os.path.join(DATA_PATH, action)
    video_folders = os.listdir(action_folder)

    for video_folder in video_folders:
        video_path = os.path.join(action_folder, video_folder)
        frames = sorted([os.path.join(video_path, f) for f in os.listdir(video_path) if f.endswith('.npy')], key=lambda x: int(x.split('/')[-1].split('.')[0]))
        if len(frames) == sequence_length:
            video_data = np.array([np.load(frame) for frame in frames])
            data.append(video_data)
            labels.append(action)

# 레이블을 숫자로 변환 및 원-핫 인코딩
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

X = np.array(data)
y = np.array(labels)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 정의
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(sequence_length, 258)),
    MaxPooling1D(2),
    Conv1D(128, 3, activation='relu'),
    MaxPooling1D(2),
    Conv1D(64, 3, activation='relu'),
    MaxPooling1D(2),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),  # L2 정규화 적용
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),  # L2 정규화 적용
    Dense(len(actions), activation='softmax')
])

# 모델 컴파일
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 학습 파라미터 설정 및 모델 학습
epochs = 50
batch_size = 10

# 학습 파라미터 설정 및 모델 학습
train_generator = DataGenerator(X_train, y_train, batch_size, shuffle=True, augment=True)
val_generator = DataGenerator(X_test, y_test, batch_size, shuffle=False)

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[EarlyStopping(monitor='accuracy', patience=10, restore_best_weights=True)]
)

# 평가 및 결과 출력
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {test_accuracy:.4f}')

Epoch 1/50
12/12 [==============================] - 3s 67ms/step - loss: 3.5577 - accuracy: 0.0583 - val_loss: 3.4248 - val_accuracy: 0.1333
Epoch 2/50
12/12 [==============================] - 0s 26ms/step - loss: 3.3719 - accuracy: 0.0917 - val_loss: 3.2936 - val_accuracy: 0.1333
Epoch 3/50
12/12 [==============================] - 0s 27ms/step - loss: 3.2268 - accuracy: 0.1250 - val_loss: 3.1535 - val_accuracy: 0.1333
Epoch 4/50
12/12 [==============================] - 0s 16ms/step - loss: 3.0986 - accuracy: 0.1250 - val_loss: 3.0514 - val_accuracy: 0.0333
Epoch 5/50
12/12 [==============================] - 0s 15ms/step - loss: 2.9997 - accuracy: 0.1083 - val_loss: 2.9573 - val_accuracy: 0.0333
Epoch 6/50
12/12 [==============================] - 0s 13ms/step - loss: 2.8975 - accuracy: 0.1333 - val_loss: 2.8661 - val_accuracy: 0.0333
Epoch 7/50
12/12 [==============================] - 0s 18ms/step - loss: 2.8049 - accuracy: 0.1833 - val_loss: 2.7827 - val_accuracy: 0.0333
Epoch 8/50
12

In [ ]:
# 모델 저장
model.save('model_.h5')

# 모델 불러오기
model = tf.keras.models.load_model('model_.h5')

# TFLite 변환기 생성
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# 변환 수행
tflite_model = converter.convert()

# TFLite 모델 파일로 저장
with open('model_5.tflite', 'wb') as f:
    f.write(tflite_model)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import load_model
import os
import numpy as np

# 저장된 모델 로드
model = load_model('model_3.h5')

# 예측을 위한 새로운 데이터 준비
new_data = []  # 새로운 데이터를 저장할 리스트
sequence_length = 30
actions = ['곰', '놀이터', '다리', '바다', '벌', '병원', '선생님', '엄마', '유치원', '학교']

# 예시: 새로운 수화 영상을 로드하여 데이터로 변환
new_video_path = '/content/drive/MyDrive/KSL_MP/바다/KETI_SL_0000014887'
new_frames = sorted([f for f in os.listdir(new_video_path) if f.endswith('.npy')], key=lambda x: int(x.split('.')[0]))
if len(new_frames) == sequence_length:
    new_video_data = np.array([np.load(os.path.join(new_video_path, frame)) for frame in new_frames])
    new_data.append(new_video_data)

# 데이터를 numpy 배열로 변환
X_new = np.array(new_data)

# 모델을 사용하여 예측
predictions = model.predict(X_new)

# 예측 결과 해석
predicted_labels = [actions[np.argmax(pred)] for pred in predictions]

# 예측 결과 출력
print("Predicted labels:", predicted_labels)

1/1 [==============================] - 0s 238ms/step
Predicted labels: ['바다']
